In [1]:
! pip install mlxtend

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 64.9 MB/s eta 0:00:00


In [138]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import seaborn as sns
from collections import defaultdict
import scipy.stats as stat
import importlib


import matplotlib.pyplot as plt
from sklearn.metrics import r2_score
from sklearn.preprocessing import MinMaxScaler,StandardScaler


from sklearn.tree import DecisionTreeRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import SGDRegressor
from sklearn.linear_model import ElasticNet
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso

from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import GridSearchCV
from mlxtend.feature_selection import SequentialFeatureSelector as SFS

from sklearn.preprocessing import FunctionTransformer
from sklearn.compose import ColumnTransformer

In [213]:
df_train=pd.read_csv("house_price_train.csv")

In [214]:
df_train.drop("Id",axis=1,inplace=True) ## reduntatnt
output_col=df_train['SalePrice']
df_train.drop('SalePrice',axis=1,inplace=True)


In [215]:
df_train

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,...,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition
0,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,Inside,...,0,0,NaN,NaN,NaN,0,2,2008,WD,Normal
1,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,FR2,...,0,0,NaN,NaN,NaN,0,5,2007,WD,Normal
2,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,Inside,...,0,0,NaN,NaN,NaN,0,9,2008,WD,Normal
3,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,Corner,...,0,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml
4,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,FR2,...,0,0,NaN,NaN,NaN,0,12,2008,WD,Normal
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1455,60,RL,62.0,7917,Pave,NaN,Reg,Lvl,AllPub,Inside,...,0,0,NaN,NaN,NaN,0,8,2007,WD,Normal
1456,20,RL,85.0,13175,Pave,NaN,Reg,Lvl,AllPub,Inside,...,0,0,NaN,MnPrv,NaN,0,2,2010,WD,Normal
1457,70,RL,66.0,9042,Pave,NaN,Reg,Lvl,AllPub,Inside,...,0,0,NaN,GdPrv,Shed,2500,5,2010,WD,Normal
1458,20,RL,68.0,9717,Pave,NaN,Reg,Lvl,AllPub,Inside,...,0,0,NaN,NaN,NaN,0,4,2010,WD,Normal


In [216]:
df_train.describe()


,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,...,GarageArea,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold
count,1460.000000,1201.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1452.000000,1460.000000,1460.000000,...,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000
mean,56.897260,70.049958,10516.828082,6.099315,5.575342,1971.267808,1984.865753,103.685262,443.639726,46.549315,...,472.980137,94.244521,46.660274,21.954110,3.409589,15.060959,2.758904,43.489041,6.321918,2007.815753
std,42.300571,24.284752,9981.264932,1.382997,1.112799,30.202904,20.645407,181.066207,456.098091,161.319273,...,213.804841,125.338794,66.256028,61.119149,29.317331,55.757415,40.177307,496.123024,2.703626,1.328095
min,20.000000,21.000000,1300.000000,1.000000,1.000000,1872.000000,1950.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,2006.000000
25%,20.000000,59.000000,7553.500000,5.000000,5.000000,1954.000000,1967.000000,0.000000,0.000000,0.000000,...,334.500000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,5.000000,2007.000000
50%,50.000000,69.000000,9478.500000,6.000000,5.000000,1973.000000,1994.000000,0.000000,383.500000,0.000000,...,480.000000,0.000000,25.000000,0.000000,0.000000,0.000000,0.000000,0.000000,6.000000,2008.000000
75%,70.000000,80.000000,11601.500000,7.000000,6.000000,2000.000000,2004.000000,166.000000,712.250000,0.000000,...,576.000000,168.000000,68.000000,0.000000,0.000000,0.000000,0.000000,0.000000,8.000000,2009.000000
max,190.000000,313.000000,215245.000000,10.000000,9.000000,2010.000000,2010.000000,1600.000000,5644.000000,1474.000000,...,1418.000000,857.000000,547.000000,552.000000,508.000000,480.000000,738.000000,15500.000000,12.000000,2010.000000


In [217]:
df_train.isnull().sum()

MSSubClass         0
MSZoning           0
LotFrontage      259
LotArea            0
Street             0
                ... 
MiscVal            0
MoSold             0
YrSold             0
SaleType           0
SaleCondition      0
Length: 79, dtype: int64

In [218]:
# Columns which have zero values in more than 50% rows are removed
def column_with_most_reduntant_values(reduntant_value,df_train:pd.DataFrame,threshold:float):
    reduntant_counts={}
    columns_with_most_reduntant_counts=[]
    for col in df_train.columns:
        if df_train[col].dtypes=='int' or df_train[col].dtypes=='float':
            for values in df_train[col]:
                if values==reduntant_value:
                    reduntant_counts[col]=1+reduntant_counts.get(col,0)
    for col in reduntant_counts:
        if reduntant_counts[col]>=(threshold*len(df_train)):
            columns_with_most_reduntant_counts.append(col)
    df_train.drop(columns_with_most_reduntant_counts,axis=1,inplace=True)
        
        
            

In [219]:
column_with_most_reduntant_values(reduntant_value=0,df_train=df_train,threshold=0.5)

In [220]:
df_train

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,...,GarageCond,PavedDrive,OpenPorchSF,PoolQC,Fence,MiscFeature,MoSold,YrSold,SaleType,SaleCondition
0,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,Inside,...,TA,Y,61,NaN,NaN,NaN,2,2008,WD,Normal
1,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,FR2,...,TA,Y,0,NaN,NaN,NaN,5,2007,WD,Normal
2,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,Inside,...,TA,Y,42,NaN,NaN,NaN,9,2008,WD,Normal
3,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,Corner,...,TA,Y,35,NaN,NaN,NaN,2,2006,WD,Abnorml
4,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,FR2,...,TA,Y,84,NaN,NaN,NaN,12,2008,WD,Normal
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1455,60,RL,62.0,7917,Pave,NaN,Reg,Lvl,AllPub,Inside,...,TA,Y,40,NaN,NaN,NaN,8,2007,WD,Normal
1456,20,RL,85.0,13175,Pave,NaN,Reg,Lvl,AllPub,Inside,...,TA,Y,0,NaN,MnPrv,NaN,2,2010,WD,Normal
1457,70,RL,66.0,9042,Pave,NaN,Reg,Lvl,AllPub,Inside,...,TA,Y,60,NaN,GdPrv,Shed,5,2010,WD,Normal
1458,20,RL,68.0,9717,Pave,NaN,Reg,Lvl,AllPub,Inside,...,TA,Y,0,NaN,NaN,NaN,4,2010,WD,Normal


In [221]:
# Deleting columns which have more than 50% NULL values

def column_with_most_reduntant_null_values(threshold,df_train:pd.DataFrame):
    null_cols=[]
    for col in df_train.columns:
        if df_train[col].isnull().sum()>=(threshold*len(df_train)):
            null_cols.append(col)
    df_train.drop(null_cols,axis=1,inplace=True)
   


In [222]:
# print(column_with_most_reduntant_null_values(threshold=0.5,X=df_train))

In [223]:
column_with_most_reduntant_null_values(threshold=0.5,df_train=df_train)

In [224]:
df_train

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,LotShape,LandContour,Utilities,LotConfig,LandSlope,...,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,OpenPorchSF,MoSold,YrSold,SaleType,SaleCondition
0,60,RL,65.0,8450,Pave,Reg,Lvl,AllPub,Inside,Gtl,...,2,548,TA,TA,Y,61,2,2008,WD,Normal
1,20,RL,80.0,9600,Pave,Reg,Lvl,AllPub,FR2,Gtl,...,2,460,TA,TA,Y,0,5,2007,WD,Normal
2,60,RL,68.0,11250,Pave,IR1,Lvl,AllPub,Inside,Gtl,...,2,608,TA,TA,Y,42,9,2008,WD,Normal
3,70,RL,60.0,9550,Pave,IR1,Lvl,AllPub,Corner,Gtl,...,3,642,TA,TA,Y,35,2,2006,WD,Abnorml
4,60,RL,84.0,14260,Pave,IR1,Lvl,AllPub,FR2,Gtl,...,3,836,TA,TA,Y,84,12,2008,WD,Normal
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1455,60,RL,62.0,7917,Pave,Reg,Lvl,AllPub,Inside,Gtl,...,2,460,TA,TA,Y,40,8,2007,WD,Normal
1456,20,RL,85.0,13175,Pave,Reg,Lvl,AllPub,Inside,Gtl,...,2,500,TA,TA,Y,0,2,2010,WD,Normal
1457,70,RL,66.0,9042,Pave,Reg,Lvl,AllPub,Inside,Gtl,...,1,252,TA,TA,Y,60,5,2010,WD,Normal
1458,20,RL,68.0,9717,Pave,Reg,Lvl,AllPub,Inside,Gtl,...,1,240,TA,TA,Y,0,4,2010,WD,Normal


In [225]:
i=0
for cols in df_train.columns:
    if df_train[cols].dtype=="int64":
        # print(cols,i)
        i+=1
    elif df_train[cols].dtype=="object":
#         print(cols,i)
        i+=1
#     elif df_train[cols].dtype=="float64":
#         print(cols,i)
# #         i+=1
print(i)

60


In [226]:
df_train

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,LotShape,LandContour,Utilities,LotConfig,LandSlope,...,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,OpenPorchSF,MoSold,YrSold,SaleType,SaleCondition
0,60,RL,65.0,8450,Pave,Reg,Lvl,AllPub,Inside,Gtl,...,2,548,TA,TA,Y,61,2,2008,WD,Normal
1,20,RL,80.0,9600,Pave,Reg,Lvl,AllPub,FR2,Gtl,...,2,460,TA,TA,Y,0,5,2007,WD,Normal
2,60,RL,68.0,11250,Pave,IR1,Lvl,AllPub,Inside,Gtl,...,2,608,TA,TA,Y,42,9,2008,WD,Normal
3,70,RL,60.0,9550,Pave,IR1,Lvl,AllPub,Corner,Gtl,...,3,642,TA,TA,Y,35,2,2006,WD,Abnorml
4,60,RL,84.0,14260,Pave,IR1,Lvl,AllPub,FR2,Gtl,...,3,836,TA,TA,Y,84,12,2008,WD,Normal
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1455,60,RL,62.0,7917,Pave,Reg,Lvl,AllPub,Inside,Gtl,...,2,460,TA,TA,Y,40,8,2007,WD,Normal
1456,20,RL,85.0,13175,Pave,Reg,Lvl,AllPub,Inside,Gtl,...,2,500,TA,TA,Y,0,2,2010,WD,Normal
1457,70,RL,66.0,9042,Pave,Reg,Lvl,AllPub,Inside,Gtl,...,1,252,TA,TA,Y,60,5,2010,WD,Normal
1458,20,RL,68.0,9717,Pave,Reg,Lvl,AllPub,Inside,Gtl,...,1,240,TA,TA,Y,0,4,2010,WD,Normal


In [227]:
# Filling Null Values

def fill_null_values(for_numerical_cols:str,for_categorical_cols:str,df_train):
    for cols in df_train.columns:
        if df_train[cols].dtype=="int64" or df_train[cols].dtype=="float64":
            if for_numerical_cols=="median":
                median_=df_train[cols].median()
                df_train[cols].fillna(median_,inplace=True)
        elif df_train[cols].dtype=="object":
            if for_categorical_cols=="most_frequent":
                d=list(df_train[cols].value_counts().index) # most frequent
                df_train[cols].fillna(d[0],inplace=True)


In [228]:
fill_null_values(for_numerical_cols="median",for_categorical_cols="most_frequent",df_train=df_train)

In [229]:
for col in df_train.columns:
    if df_train[col].isnull().sum()>(0):

        print(col,df[col].isnull().sum(),df_train[col].dtypes)


In [230]:
df_train

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,LotShape,LandContour,Utilities,LotConfig,LandSlope,...,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,OpenPorchSF,MoSold,YrSold,SaleType,SaleCondition
0,60,RL,65.0,8450,Pave,Reg,Lvl,AllPub,Inside,Gtl,...,2,548,TA,TA,Y,61,2,2008,WD,Normal
1,20,RL,80.0,9600,Pave,Reg,Lvl,AllPub,FR2,Gtl,...,2,460,TA,TA,Y,0,5,2007,WD,Normal
2,60,RL,68.0,11250,Pave,IR1,Lvl,AllPub,Inside,Gtl,...,2,608,TA,TA,Y,42,9,2008,WD,Normal
3,70,RL,60.0,9550,Pave,IR1,Lvl,AllPub,Corner,Gtl,...,3,642,TA,TA,Y,35,2,2006,WD,Abnorml
4,60,RL,84.0,14260,Pave,IR1,Lvl,AllPub,FR2,Gtl,...,3,836,TA,TA,Y,84,12,2008,WD,Normal
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1455,60,RL,62.0,7917,Pave,Reg,Lvl,AllPub,Inside,Gtl,...,2,460,TA,TA,Y,40,8,2007,WD,Normal
1456,20,RL,85.0,13175,Pave,Reg,Lvl,AllPub,Inside,Gtl,...,2,500,TA,TA,Y,0,2,2010,WD,Normal
1457,70,RL,66.0,9042,Pave,Reg,Lvl,AllPub,Inside,Gtl,...,1,252,TA,TA,Y,60,5,2010,WD,Normal
1458,20,RL,68.0,9717,Pave,Reg,Lvl,AllPub,Inside,Gtl,...,1,240,TA,TA,Y,0,4,2010,WD,Normal


In [231]:
continuous_cols=list(df_train.select_dtypes(include= np.number).columns)

In [232]:
categorical_cols=list(df_train.select_dtypes(exclude= np.number).columns)

In [233]:
categorical_cols

['MSZoning',
 'Street',
 'LotShape',
 'LandContour',
 'Utilities',
 'LotConfig',
 'LandSlope',
 'Neighborhood',
 'Condition1',
 'Condition2',
 'BldgType',
 'HouseStyle',
 'RoofStyle',
 'RoofMatl',
 'Exterior1st',
 'Exterior2nd',
 'MasVnrType',
 'ExterQual',
 'ExterCond',
 'Foundation',
 'BsmtQual',
 'BsmtCond',
 'BsmtExposure',
 'BsmtFinType1',
 'BsmtFinType2',
 'Heating',
 'HeatingQC',
 'CentralAir',
 'Electrical',
 'KitchenQual',
 'Functional',
 'FireplaceQu',
 'GarageType',
 'GarageFinish',
 'GarageQual',
 'GarageCond',
 'PavedDrive',
 'SaleType',
 'SaleCondition']

In [234]:
df_train["MSZoning"].dtypes

dtype('O')

In [235]:
# fig, axes = plt.subplots(nrows = 6, ncols = 4)    # axes is 2d array (3x3)
# axes = axes.flatten()         # Convert axes to 1d array of length 9
# fig.set_size_inches(40, 40)

# for ax, col in zip(axes, continuous_cols):
#     print()
#     sns.distplot(df_train[col], ax = ax,kde=True)
#     ax.set_title(col)

In [236]:
# fig, axes = plt.subplots(nrows = 6, ncols = 4)#   skew axes is 2d array (3x3)
# axes = axes.flatten()         # Convert axes to 1d array of length 9
# fig.set_size_inches(40, 40)

# for ax, col in zip(axes, continuous_cols):
#     # print()
#     sns.boxplot(df_train[col], ax = ax)
#     ax.set_title(col)

In [237]:
# plt.figure(figsize=(20,15))
# sns.heatmap(df_train.loc[:,continuous_cols].corr(method='spearman'),annot=True,cmap="RdYlGn")
# plt.show()

In [238]:
# sns.set(rc={'figure.figsize':(40,40)})
# cData_attr = df_train.loc[:,continuous_cols]
# sns.pairplot(cData_attr, diag_kind='kde')  

In [239]:
df_train.loc[:,continuous_cols]

,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,BsmtFinSF1,BsmtUnfSF,TotalBsmtSF,...,BedroomAbvGr,KitchenAbvGr,TotRmsAbvGrd,Fireplaces,GarageYrBlt,GarageCars,GarageArea,OpenPorchSF,MoSold,YrSold
0,60,65.0,8450,7,5,2003,2003,706,150,856,...,3,1,8,0,2003.0,2,548,61,2,2008
1,20,80.0,9600,6,8,1976,1976,978,284,1262,...,3,1,6,1,1976.0,2,460,0,5,2007
2,60,68.0,11250,7,5,2001,2002,486,434,920,...,3,1,6,1,2001.0,2,608,42,9,2008
3,70,60.0,9550,7,5,1915,1970,216,540,756,...,3,1,7,1,1998.0,3,642,35,2,2006
4,60,84.0,14260,8,5,2000,2000,655,490,1145,...,4,1,9,1,2000.0,3,836,84,12,2008
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1455,60,62.0,7917,6,5,1999,2000,0,953,953,...,3,1,7,1,1999.0,2,460,40,8,2007
1456,20,85.0,13175,6,6,1978,1988,790,589,1542,...,3,1,7,2,1978.0,2,500,0,2,2010
1457,70,66.0,9042,7,9,1941,2006,275,877,1152,...,4,1,9,2,1941.0,1,252,60,5,2010
1458,20,68.0,9717,5,6,1950,1996,49,0,1078,...,2,1,5,0,1950.0,1,240,0,4,2010


In [240]:
continuous_cols

['MSSubClass',
 'LotFrontage',
 'LotArea',
 'OverallQual',
 'OverallCond',
 'YearBuilt',
 'YearRemodAdd',
 'BsmtFinSF1',
 'BsmtUnfSF',
 'TotalBsmtSF',
 '1stFlrSF',
 'GrLivArea',
 'FullBath',
 'BedroomAbvGr',
 'KitchenAbvGr',
 'TotRmsAbvGrd',
 'Fireplaces',
 'GarageYrBlt',
 'GarageCars',
 'GarageArea',
 'OpenPorchSF',
 'MoSold',
 'YrSold']

In [241]:
# df_train=pd.concat([output_col,df_train],axis=1) # since rows will be deleted here


In [242]:

# for col in continuous_cols:
#     q1=df_train[col].quantile(0.25)
#     q3=df_train[col].quantile(0.75)
#     iqr=q3-q1
#     l=q1-1.5*iqr
#     h=q3+1.5*iqr
#     print("[",l,",",h,"]")
#     df_train = df_train[(df_train[col] <= h)]# & (df_train[col] >=l)] 
#     df_train =df_train[(df_train[col] >=l)] 




In [243]:
# output_col=df_train['SalePrice']
# df_train.drop("SalePrice",axis=1,inplace=True)

In [244]:
df_train

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,LotShape,LandContour,Utilities,LotConfig,LandSlope,...,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,OpenPorchSF,MoSold,YrSold,SaleType,SaleCondition
0,60,RL,65.0,8450,Pave,Reg,Lvl,AllPub,Inside,Gtl,...,2,548,TA,TA,Y,61,2,2008,WD,Normal
1,20,RL,80.0,9600,Pave,Reg,Lvl,AllPub,FR2,Gtl,...,2,460,TA,TA,Y,0,5,2007,WD,Normal
2,60,RL,68.0,11250,Pave,IR1,Lvl,AllPub,Inside,Gtl,...,2,608,TA,TA,Y,42,9,2008,WD,Normal
3,70,RL,60.0,9550,Pave,IR1,Lvl,AllPub,Corner,Gtl,...,3,642,TA,TA,Y,35,2,2006,WD,Abnorml
4,60,RL,84.0,14260,Pave,IR1,Lvl,AllPub,FR2,Gtl,...,3,836,TA,TA,Y,84,12,2008,WD,Normal
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1455,60,RL,62.0,7917,Pave,Reg,Lvl,AllPub,Inside,Gtl,...,2,460,TA,TA,Y,40,8,2007,WD,Normal
1456,20,RL,85.0,13175,Pave,Reg,Lvl,AllPub,Inside,Gtl,...,2,500,TA,TA,Y,0,2,2010,WD,Normal
1457,70,RL,66.0,9042,Pave,Reg,Lvl,AllPub,Inside,Gtl,...,1,252,TA,TA,Y,60,5,2010,WD,Normal
1458,20,RL,68.0,9717,Pave,Reg,Lvl,AllPub,Inside,Gtl,...,1,240,TA,TA,Y,0,4,2010,WD,Normal


In [245]:
categorical_cols

['MSZoning',
 'Street',
 'LotShape',
 'LandContour',
 'Utilities',
 'LotConfig',
 'LandSlope',
 'Neighborhood',
 'Condition1',
 'Condition2',
 'BldgType',
 'HouseStyle',
 'RoofStyle',
 'RoofMatl',
 'Exterior1st',
 'Exterior2nd',
 'MasVnrType',
 'ExterQual',
 'ExterCond',
 'Foundation',
 'BsmtQual',
 'BsmtCond',
 'BsmtExposure',
 'BsmtFinType1',
 'BsmtFinType2',
 'Heating',
 'HeatingQC',
 'CentralAir',
 'Electrical',
 'KitchenQual',
 'Functional',
 'FireplaceQu',
 'GarageType',
 'GarageFinish',
 'GarageQual',
 'GarageCond',
 'PavedDrive',
 'SaleType',
 'SaleCondition']

In [246]:


def encoding_categorical_cols(df_train:pd.DataFrame,categorical_columns:list,strategy:str):
    df_categorical=pd.DataFrame(df_train.loc[:,categorical_columns])
    if strategy=="labelencoder":
        for col in list(df_categorical.columns):
            cat=[]
            d={}

            cat.append([df_categorical[col].value_counts().index,len(df_categorical[col].value_counts().index)])

            for j in range(int(cat[0][1])):
                d[str(cat[0][0][j])]=j
            df_categorical[col]=df_categorical[col].map(d)
        df_train.drop(categorical_columns,axis=1,inplace=True)
        
        df_train=pd.concat([df_train,df_categorical],axis=1)
        
    if strategy=="onehotencoder":
        one_hot_encoded=pd.get_dummies(df_categorical)
        # print(one_hot_encoded)
        df_train.drop(categorical_columns,axis=1,inplace=True)
        df_train=pd.concat([df_train,one_hot_encoded],axis=1)
    return df_train
        

    
    


One hot encoding since hyperparameter tuning does not have its effects

In [247]:
df_train=encoding_categorical_cols(df_train=df_train,categorical_columns=categorical_cols,strategy="onehotencoder")

In [248]:
df_train

,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,BsmtFinSF1,BsmtUnfSF,TotalBsmtSF,...,KitchenQual,Functional,FireplaceQu,GarageType,GarageFinish,GarageQual,GarageCond,PavedDrive,SaleType,SaleCondition
0,60,65.0,8450,7,5,2003,2003,706,150,856,...,1,0,0,0,1,0,0,0,0,0
1,20,80.0,9600,6,8,1976,1976,978,284,1262,...,0,0,1,0,1,0,0,0,0,0
2,60,68.0,11250,7,5,2001,2002,486,434,920,...,1,0,1,0,1,0,0,0,0,0
3,70,60.0,9550,7,5,1915,1970,216,540,756,...,1,0,0,1,0,0,0,0,0,2
4,60,84.0,14260,8,5,2000,2000,655,490,1145,...,1,0,1,0,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1455,60,62.0,7917,6,5,1999,2000,0,953,953,...,0,0,1,0,1,0,0,0,0,0
1456,20,85.0,13175,6,6,1978,1988,790,589,1542,...,0,2,1,0,0,0,0,0,0,0
1457,70,66.0,9042,7,9,1941,2006,275,877,1152,...,1,0,0,0,1,0,0,0,0,0
1458,20,68.0,9717,5,6,1950,1996,49,0,1078,...,1,0,0,0,0,0,0,0,0,0


In [249]:
for col in categorical_cols:
    if col in set(df_train.columns):
        print(col)

MSZoning
Street
LotShape
LandContour
Utilities
LotConfig
LandSlope
Neighborhood
Condition1
Condition2
BldgType
HouseStyle
RoofStyle
RoofMatl
Exterior1st
Exterior2nd
MasVnrType
ExterQual
ExterCond
Foundation
BsmtQual
BsmtCond
BsmtExposure
BsmtFinType1
BsmtFinType2
Heating
HeatingQC
CentralAir
Electrical
KitchenQual
Functional
FireplaceQu
GarageType
GarageFinish
GarageQual
GarageCond
PavedDrive
SaleType
SaleCondition


In [250]:
df_train.loc[:,continuous_cols].describe()

,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,BsmtFinSF1,BsmtUnfSF,TotalBsmtSF,...,BedroomAbvGr,KitchenAbvGr,TotRmsAbvGrd,Fireplaces,GarageYrBlt,GarageCars,GarageArea,OpenPorchSF,MoSold,YrSold
count,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,...,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000
mean,56.897260,69.863699,10516.828082,6.099315,5.575342,1971.267808,1984.865753,443.639726,567.240411,1057.429452,...,2.866438,1.046575,6.517808,0.613014,1978.589041,1.767123,472.980137,46.660274,6.321918,2007.815753
std,42.300571,22.027677,9981.264932,1.382997,1.112799,30.202904,20.645407,456.098091,441.866955,438.705324,...,0.815778,0.220338,1.625393,0.644666,23.997022,0.747315,213.804841,66.256028,2.703626,1.328095
min,20.000000,21.000000,1300.000000,1.000000,1.000000,1872.000000,1950.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,2.000000,0.000000,1900.000000,0.000000,0.000000,0.000000,1.000000,2006.000000
25%,20.000000,60.000000,7553.500000,5.000000,5.000000,1954.000000,1967.000000,0.000000,223.000000,795.750000,...,2.000000,1.000000,5.000000,0.000000,1962.000000,1.000000,334.500000,0.000000,5.000000,2007.000000
50%,50.000000,69.000000,9478.500000,6.000000,5.000000,1973.000000,1994.000000,383.500000,477.500000,991.500000,...,3.000000,1.000000,6.000000,1.000000,1980.000000,2.000000,480.000000,25.000000,6.000000,2008.000000
75%,70.000000,79.000000,11601.500000,7.000000,6.000000,2000.000000,2004.000000,712.250000,808.000000,1298.250000,...,3.000000,1.000000,7.000000,1.000000,2001.000000,2.000000,576.000000,68.000000,8.000000,2009.000000
max,190.000000,313.000000,215245.000000,10.000000,9.000000,2010.000000,2010.000000,5644.000000,2336.000000,6110.000000,...,8.000000,3.000000,14.000000,3.000000,2010.000000,4.000000,1418.000000,547.000000,12.000000,2010.000000


In [251]:
### SCALING

In [252]:
scaler=scaler=MinMaxScaler()
df_train=pd.DataFrame(scaler.fit_transform(df_train),columns=df_train.columns,)
df_train

,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,BsmtFinSF1,BsmtUnfSF,TotalBsmtSF,...,KitchenQual,Functional,FireplaceQu,GarageType,GarageFinish,GarageQual,GarageCond,PavedDrive,SaleType,SaleCondition
0,0.235294,0.150685,0.033420,0.666667,0.500,0.949275,0.883333,0.125089,0.064212,0.140098,...,0.333333,0.000000,0.00,0.0,0.5,0.0,0.0,0.0,0.0,0.0
1,0.000000,0.202055,0.038795,0.555556,0.875,0.753623,0.433333,0.173281,0.121575,0.206547,...,0.000000,0.000000,0.25,0.0,0.5,0.0,0.0,0.0,0.0,0.0
2,0.235294,0.160959,0.046507,0.666667,0.500,0.934783,0.866667,0.086109,0.185788,0.150573,...,0.333333,0.000000,0.25,0.0,0.5,0.0,0.0,0.0,0.0,0.0
3,0.294118,0.133562,0.038561,0.666667,0.500,0.311594,0.333333,0.038271,0.231164,0.123732,...,0.333333,0.000000,0.00,0.2,0.0,0.0,0.0,0.0,0.0,0.4
4,0.235294,0.215753,0.060576,0.777778,0.500,0.927536,0.833333,0.116052,0.209760,0.187398,...,0.333333,0.000000,0.25,0.0,0.5,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1455,0.235294,0.140411,0.030929,0.555556,0.500,0.920290,0.833333,0.000000,0.407962,0.155974,...,0.000000,0.000000,0.25,0.0,0.5,0.0,0.0,0.0,0.0,0.0
1456,0.000000,0.219178,0.055505,0.555556,0.625,0.768116,0.633333,0.139972,0.252140,0.252373,...,0.000000,0.333333,0.25,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1457,0.294118,0.154110,0.036187,0.666667,1.000,0.500000,0.933333,0.048724,0.375428,0.188543,...,0.333333,0.000000,0.00,0.0,0.5,0.0,0.0,0.0,0.0,0.0
1458,0.000000,0.160959,0.039342,0.444444,0.625,0.565217,0.766667,0.008682,0.000000,0.176432,...,0.333333,0.000000,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [253]:
from sklearn.metrics import r2_score

In [254]:
X=df_train
Y=output_col

In [255]:
X_train,X_test,Y_train,Y_test=train_test_split(X,Y,test_size=0.2,random_state=None,shuffle=True)


In [256]:
accuracies={}

Atlast we will be doing manual hyperparameter tuning. Firstly we wiil do Random SearchCV since it gives us a region in the form of parameters where our accuracy could we very high,this reduces our search space. Then using those parameters we will use Grid Search CV to get best result using combinations of those parameters.

In [257]:
def hyperparameter_tuning(algo:str,param_grid:dict,n_iter:int,cv:int,verbose:int,random_state:int,X_train,Y_train,X_test,Y_test,evaluation_metric:str):
    model=eval(algo)
    evaluation_metric=eval(evaluation_metric)
    
    model_random_cv=RandomizedSearchCV(estimator=model,param_distributions=param_grid,n_iter=n_iter,cv=cv,verbose=verbose,random_state=random_state,n_jobs=-1)
    model_random_cv.fit(X_train,Y_train)
    best_params_=model_random_cv.best_params_
    print(f"{model} RandomCV Best Params :",best_params_)
    best_random_grid=model_random_cv.best_estimator_
    y_pred=best_random_grid.predict(X_test)
    
    score=evaluation_metric(Y_test,y_pred)
    print(f"{model} RandomCV Score:",score)
    
    
    
    param_grid=defaultdict(list)
    for val in best_params_:
        param_grid[val].append(best_params_[val])
    
    
    model_for_gcv=eval(algo)
    
    grid_search=GridSearchCV(estimator=model_for_gcv,param_grid=param_grid,cv=cv,n_jobs=-1,verbose=verbose)
    grid_search.fit(X_train,Y_train)
    best_grid2=grid_search.best_estimator_
    y_pred2=best_grid2.predict(X_test)
    score_1=evaluation_metric(Y_test,y_pred2)
    print("Final score is ",score_1)
    accuracies[algo]=score_1
    
    
   

In [258]:
## On Linear Regresion


In [259]:
# On Linear Regresion
fit_intercept = ["True","False"]
copy_X =["True","False"] 
positive=["False","True"]

random_grid = {'fit_intercept': fit_intercept,
               'positive': positive,
               'copy_X': copy_X}
print(random_grid)

{'fit_intercept': ['True', 'False'], 'positive': ['False', 'True'], 'copy_X': ['True', 'False']}


In [260]:
hyperparameter_tuning(algo="LinearRegression()",param_grid=random_grid,n_iter=100,cv=5,verbose=3,random_state=42,X_train=X_train,Y_train=Y_train,X_test=X_test,Y_test=Y_test,evaluation_metric="r2_score")

Fitting 5 folds for each of 8 candidates, totalling 40 fits
LinearRegression() RandomCV Best Params : {'positive': 'False', 'fit_intercept': 'True', 'copy_X': 'True'}
LinearRegression() RandomCV Score: 0.7518561285635769
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Final score is  0.7518561285635769


/usr/local/lib/python3.9/dist-packages/sklearn/model_selection/_search.py:306: UserWarning: The total space of parameters 8 is smaller than n_iter=100. Running 8 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


In [ ]:

# lr=LinearRegression()
# lr_random_cv=RandomizedSearchCV(estimator=lr,param_distributions=random_grid,n_iter=100,cv=5,verbose=0,random_state=100,n_jobs=-1)
# lr_random_cv.fit(X_train,Y_train)
# best_params_=lr_random_cv.best_params_
# print("RandomCV Best Params :",best_params_)
# best_random_grid=lr_random_cv.best_estimator_
# y_pred1=best_random_grid.predict(X_test)
# score_1=r2_score(Y_test,y_pred1)
# print("RandomCV Score: ",score_1)

In [ ]:
# param_grid2=defaultdict(list)
# for val in best_params_:
#     param_grid2[val].append(best_params_[val])



In [ ]:
# lr=LinearRegression()
# grid_search=GridSearchCV(estimator=lr,param_grid=param_grid2,cv=5,n_jobs=-1,verbose=3)
# grid_search.fit(X_train,Y_train)
# print(grid_search.best_estimator_)

In [ ]:
# best_grid2=grid_search.best_estimator_
# y_pred2=best_grid2.predict(X_test)
# score_1=r2_score(Y_test,y_pred2)
# print(score_1)
# accuracies["LinearRegression"]=score_1

In [229]:
# RandomForestRegressor

In [ ]:
# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 200, stop = 2000, num = 10)]
# Number of features to consider at every split
max_features = ['sqrt','log2']
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(10, 1000,10)]
# Minimum number of samples required to split a node
min_samples_split = [2,3,4, 5, 10,14]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4,6,8]
# Create the random grid
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
              'criterion':['squared_error','absolute_error','friedman_mse','poisson']}
print(random_grid)

{'n_estimators': [200, 400, 600, 800, 1000, 1200, 1400, 1600, 1800, 2000], 'max_features': ['sqrt', 'log2'], 'max_depth': [10, 20, 30, 40, 50, 60, 70, 80, 90, 100, 110, 120, 130, 140, 150, 160, 170, 180, 190, 200, 210, 220, 230, 240, 250, 260, 270, 280, 290, 300, 310, 320, 330, 340, 350, 360, 370, 380, 390, 400, 410, 420, 430, 440, 450, 460, 470, 480, 490, 500, 510, 520, 530, 540, 550, 560, 570, 580, 590, 600, 610, 620, 630, 640, 650, 660, 670, 680, 690, 700, 710, 720, 730, 740, 750, 760, 770, 780, 790, 800, 810, 820, 830, 840, 850, 860, 870, 880, 890, 900, 910, 920, 930, 940, 950, 960, 970, 980, 990, 1000], 'min_samples_split': [2, 7, 12, 17, 22, 27, 32, 37, 42, 47, 52, 57, 62, 67, 72, 77, 82, 87, 92, 97, 102, 107, 112, 117, 122, 127, 132, 137, 142, 147, 152, 157, 162, 168, 173, 178, 183, 188, 193, 198, 203, 208, 213, 218, 223, 228, 233, 238, 243, 248, 253, 258, 263, 268, 273, 278, 283, 288, 293, 298, 303, 308, 313, 318, 323, 328, 334, 339, 344, 349, 354, 359, 364, 369, 374, 379, 384,

In [ ]:
hyperparameter_tuning(algo="RandomForestRegressor()",param_grid=random_grid,n_iter=10,cv=5,verbose=3,random_state=42,X_train=X_train,Y_train=Y_train,X_test=X_test,Y_test=Y_test,evaluation_metric="r2_score")

Fitting 5 folds for each of 10 candidates, totalling 50 fits
RandomForestRegressor() RandomCV Best Params : {'n_estimators': 600, 'min_samples_split': 47, 'min_samples_leaf': 77, 'max_features': 'log2', 'max_depth': 670, 'criterion': 'squared_error'}
RandomForestRegressor() RandomCV Score: 0.66952666471689
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Final score is  0.6663155322785399


In [236]:
# 3. SVR

In [261]:
kernel = ["linear", "poly", "sigmoid","poly"]
degree = [int(x) for x in np.linspace(start = 1, stop = 200, num = 10)]
gamma= ["scale", "auto"]

tol=[1e-2]
max_iter=[100]


random_grid = {'kernel': kernel,
               'degree':degree,
               'max_iter':max_iter,
               'tol':tol,
               'gamma':gamma
               
               
               }
print(random_grid)

{'kernel': ['linear', 'poly', 'sigmoid', 'poly'], 'degree': [1, 23, 45, 67, 89, 111, 133, 155, 177, 200], 'max_iter': [100], 'tol': [0.01], 'gamma': ['scale', 'auto']}


In [262]:
hyperparameter_tuning(algo="SVR()",param_grid=random_grid,n_iter=10,cv=5,verbose=0,random_state=42,X_train=X_train,Y_train=Y_train,X_test=X_test,Y_test=Y_test,evaluation_metric="r2_score")

/usr/local/lib/python3.9/dist-packages/sklearn/svm/_base.py:301: ConvergenceWarning: Solver terminated early (max_iter=100).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(
/usr/local/lib/python3.9/dist-packages/sklearn/svm/_base.py:301: ConvergenceWarning: Solver terminated early (max_iter=100).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(
/usr/local/lib/python3.9/dist-packages/sklearn/svm/_base.py:301: ConvergenceWarning: Solver terminated early (max_iter=100).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(
/usr/local/lib/python3.9/dist-packages/sklearn/svm/_base.py:301: ConvergenceWarning: Solver terminated early (max_iter=100).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(
/usr/local/lib/python3.9/dist-packages/sklearn/svm/_base.py:301: ConvergenceWarning: Solver terminated early (max_iter=100).  Consider pre-proce

SVR() RandomCV Best Params : {'tol': 0.01, 'max_iter': 100, 'kernel': 'poly', 'gamma': 'auto', 'degree': 67}
SVR() RandomCV Score: -0.10726984424962471
Final score is  -0.10726984424962471


/usr/local/lib/python3.9/dist-packages/sklearn/svm/_base.py:301: ConvergenceWarning: Solver terminated early (max_iter=100).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(
/usr/local/lib/python3.9/dist-packages/sklearn/svm/_base.py:301: ConvergenceWarning: Solver terminated early (max_iter=100).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(
/usr/local/lib/python3.9/dist-packages/sklearn/svm/_base.py:301: ConvergenceWarning: Solver terminated early (max_iter=100).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(
/usr/local/lib/python3.9/dist-packages/sklearn/svm/_base.py:301: ConvergenceWarning: Solver terminated early (max_iter=100).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(
/usr/local/lib/python3.9/dist-packages/sklearn/svm/_base.py:301: ConvergenceWarning: Solver terminated early (max_iter=100).  Consider pre-proce

[CV 4/5] END criterion=poisson, max_depth=290, max_features=sqrt, min_samples_leaf=414, min_samples_split=434, n_estimators=1600;, score=-0.004 total time=   1.5s
[CV 2/5] END criterion=friedman_mse, max_depth=840, max_features=log2, min_samples_leaf=278, min_samples_split=17, n_estimators=1400;, score=0.265 total time=   1.5s
[CV 1/5] END criterion=absolute_error, max_depth=60, max_features=sqrt, min_samples_leaf=409, min_samples_split=394, n_estimators=1800;, score=-0.047 total time=   2.1s
[CV 5/5] END criterion=squared_error, max_depth=670, max_features=log2, min_samples_leaf=77, min_samples_split=47, n_estimators=600;, score=0.626 total time=   0.7s
[CV 1/5] END criterion=absolute_error, max_depth=340, max_features=log2, min_samples_leaf=178, min_samples_split=419, n_estimators=200;, score=0.316 total time=   0.5s
[CV 2/5] END criterion=absolute_error, max_depth=590, max_features=sqrt, min_samples_leaf=162, min_samples_split=243, n_estimators=800;, score=0.367 total time=   2.6s
[

In [239]:
# svr=SVR()
# svr_random_cv=RandomizedSearchCV(estimator=svr,param_distributions=random_grid,n_iter=100,cv=5,verbose=0,random_state=100,n_jobs=-1)
# svr_random_cv.fit(X_train,Y_train)

In [240]:

# best_params_=svr_random_cv.best_params_
# print("RandomCV Best Params :",best_params_)
# best_random_grid=svr_random_cv.best_estimator_
# y_pred3=best_random_grid.predict(X_test)
# score_3=r2_score(Y_test,y_pred3)
# print("RandomCV Score: ",score_3)

In [241]:
# param_grid3=defaultdict(list)
# for val in best_params_:
#     param_grid3[val].append(best_params_[val])
# print(param_grid3)

In [242]:
# svr=SVR()
# grid_search_svr=GridSearchCV(estimator=svr,param_grid=param_grid3,cv=5,n_jobs=-1,verbose=3)
# grid_search_svr.fit(X_train,Y_train)
# print(grid_search_svr.best_estimator_)

In [243]:
# best_grid2=grid_search_svr.best_estimator_
# y_pred3=best_grid2.predict(X_test)
# score_3=r2_score(Y_test,y_pred3)
# print(score_3)
# accuracies["SVR"]=score_3

In [244]:
# 4 KNN Regressor

In [245]:
weights=["uniform", "distance"]
n_neighbors = [int(x) for x in np.linspace(start = 5, stop = 350, num = 50)]
algorithm= ["auto", "ball_tree","kd_tree","brute"]
leaf_size=[int(x) for x in np.linspace(start = 5, stop = 20, num = 10)]
p = [1,2]


random_grid = {'weights': weights,
               'n_neighbors': n_neighbors,
               'algorithm': algorithm,
               'leaf_size':leaf_size,
               'p':p
               }
print(random_grid)

{'weights': ['uniform', 'distance'], 'n_neighbors': [5, 12, 19, 26, 33, 40, 47, 54, 61, 68, 75, 82, 89, 96, 103, 110, 117, 124, 131, 138, 145, 152, 159, 166, 173, 181, 188, 195, 202, 209, 216, 223, 230, 237, 244, 251, 258, 265, 272, 279, 286, 293, 300, 307, 314, 321, 328, 335, 342, 350], 'algorithm': ['auto', 'ball_tree', 'kd_tree', 'brute'], 'leaf_size': [5, 6, 8, 10, 11, 13, 15, 16, 18, 20], 'p': [1, 2]}


In [246]:
hyperparameter_tuning(algo="KNeighborsRegressor()",param_grid=random_grid,n_iter=10,cv=5,verbose=2,random_state=42,X_train=X_train,Y_train=Y_train,X_test=X_test,Y_test=Y_test,evaluation_metric="r2_score")

Fitting 5 folds for each of 10 candidates, totalling 50 fits
KNeighborsRegressor() RandomCV Best Params : {'weights': 'distance', 'p': 2, 'n_neighbors': 5, 'leaf_size': 18, 'algorithm': 'brute'}
KNeighborsRegressor() RandomCV Score: 0.7073273687494936
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Final score is  0.7073273687494936


In [247]:
# model=KNeighborsRegressor()
# knn_random_cv=RandomizedSearchCV(estimator=model,param_distributions=random_grid,n_iter=100,cv=5,verbose=1,random_state=100,n_jobs=-1)
# knn_random_cv.fit(X_train,Y_train)
# best_params_=knn_random_cv.best_params_
# print("RandomCV Best Params :",best_params_)
# best_random_grid=knn_random_cv.best_estimator_
# y_pred4=best_random_grid.predict(X_test)
# score_4=r2_score(Y_test,y_pred4)
# print("RandomCV Score: ",score_4)

In [248]:
# param_grid4=defaultdict(list)
# for val in best_params_:
#     param_grid4[val].append(best_params_[val])
# print(param_grid4)

In [249]:
# model=KNeighborsRegressor()
# grid_search=GridSearchCV(estimator=model,param_grid=param_grid4,cv=5,n_jobs=-1,verbose=3)
# grid_search.fit(X_train,Y_train)
# print(grid_search.best_estimator_)

In [250]:
# best_grid2=grid_search.best_estimator_
# final=best_grid2.predict(X_test)
# score_final4=r2_score(Y_test,final)
# print(score_final4)
# accuracies["KNeighborsRegressor"]=score_final4

In [251]:
## 5 DecisionTreeRegressor

In [252]:
splitter=["best", "random"]
# Number of features to consider at every split
max_features = ['sqrt','log2']
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(10, 1000,100)]
# Minimum number of samples required to split a node
min_samples_split = [2,3,4, 5, 10,14]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1,2, 4,6,8]
# Create the random grid
random_grid = {'splitter':splitter,
               'ccp_alpha':[1,0],
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
              'criterion':['squared_error','absolute_error','friedman_mse','poisson']}
print(random_grid)

{'splitter': ['best', 'random'], 'ccp_alpha': [1, 0], 'max_features': ['sqrt', 'log2'], 'max_depth': [10, 20, 30, 40, 50, 60, 70, 80, 90, 100, 110, 120, 130, 140, 150, 160, 170, 180, 190, 200, 210, 220, 230, 240, 250, 260, 270, 280, 290, 300, 310, 320, 330, 340, 350, 360, 370, 380, 390, 400, 410, 420, 430, 440, 450, 460, 470, 480, 490, 500, 510, 520, 530, 540, 550, 560, 570, 580, 590, 600, 610, 620, 630, 640, 650, 660, 670, 680, 690, 700, 710, 720, 730, 740, 750, 760, 770, 780, 790, 800, 810, 820, 830, 840, 850, 860, 870, 880, 890, 900, 910, 920, 930, 940, 950, 960, 970, 980, 990, 1000], 'min_samples_split': [2, 3, 4, 5, 10, 14], 'min_samples_leaf': [1, 2, 4, 6, 8], 'criterion': ['squared_error', 'absolute_error', 'friedman_mse', 'poisson']}


In [253]:
hyperparameter_tuning(algo="DecisionTreeRegressor()",param_grid=random_grid,n_iter=10,cv=5,verbose=2,random_state=42,X_train=X_train,Y_train=Y_train,X_test=X_test,Y_test=Y_test,evaluation_metric="r2_score")

Fitting 5 folds for each of 10 candidates, totalling 50 fits
DecisionTreeRegressor() RandomCV Best Params : {'splitter': 'best', 'min_samples_split': 5, 'min_samples_leaf': 2, 'max_features': 'sqrt', 'max_depth': 300, 'criterion': 'poisson', 'ccp_alpha': 0}
DecisionTreeRegressor() RandomCV Score: 0.7533754964571078
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Final score is  0.6699070971484998


In [254]:
# dt=DecisionTreeRegressor()
# dt_random_cv=RandomizedSearchCV(estimator=dt,param_distributions=random_grid,n_iter=100,cv=5,verbose=0,random_state=100,n_jobs=-1)
# dt_random_cv.fit(X_train,Y_train)

# best_params_=dt_random_cv.best_params_
# print("RandomCV Best Params :",best_params_)
# best_random_grid=dt_random_cv.best_estimator_
# y_pred5=best_random_grid.predict(X_test)
# score_5=r2_score(Y_test,y_pred5)
# print("RandomCV Score: ",score_5)

In [255]:
# param_grid5=defaultdict(list)
# for val in best_params_:
#     param_grid5[val].append(best_params_[val])
# print(param_grid5)

In [256]:
# model=DecisionTreeRegressor()
# grid_search=GridSearchCV(estimator=model,param_grid=param_grid5,cv=5,n_jobs=-1,verbose=3)
# grid_search.fit(X_train,Y_train)
# print(grid_search.best_estimator_)
# best_grid2=grid_search.best_estimator_
# final5=best_grid2.predict(X_test)
# score_final5=r2_score(Y_test,final5)
# print(score_final5)
# accuracies["DecisionTreeRegressor"]=score_final5

In [257]:
## 6 SGDRegressor

In [258]:
loss = ['squared_error','huber','epsilon_insensitive','squared_epsilon_insensitive']
penalty=['l2','l1','elasticnet','None']
alpha = [float(x) for x in np.linspace(0, 0.001,50)]
l1_ratio=[float(x) for x in np.linspace(0, 0.5,50)]
max_iter=[int(x) for x in np.linspace(8000, 20000,100)]
tol=[float(x) for x in np.linspace(0.01, 0.001,100)]
learning_rate=['constant','optimal','invscaling','adaptive']
eta0=[float(x) for x in np.linspace(0.1, 0.05,100)]
early_stopping = ['True','False']
validation_fraction=[float(x) for x in np.linspace(0.1, 0.9,50)]
n_iter_no_change = [1,2, 4,6,8,5,10]
average=['bool','int']
random_grid = {
               'loss': loss,
               'penalty': penalty,
               'alpha': alpha,
               'l1_ratio': l1_ratio,
              'max_iter':max_iter,
               'tol':tol,
                'learning_rate':learning_rate,
                'eta0':eta0,
               
                'validation_fraction':validation_fraction,
                'n_iter_no_change':n_iter_no_change
                }
print(random_grid)

{'loss': ['squared_error', 'huber', 'epsilon_insensitive', 'squared_epsilon_insensitive'], 'penalty': ['l2', 'l1', 'elasticnet', 'None'], 'alpha': [0.0, 2.0408163265306123e-05, 4.0816326530612245e-05, 6.122448979591836e-05, 8.163265306122449e-05, 0.00010204081632653062, 0.00012244897959183673, 0.00014285714285714287, 0.00016326530612244898, 0.0001836734693877551, 0.00020408163265306123, 0.00022448979591836734, 0.00024489795918367346, 0.0002653061224489796, 0.00028571428571428574, 0.0003061224489795918, 0.00032653061224489796, 0.0003469387755102041, 0.0003673469387755102, 0.0003877551020408163, 0.00040816326530612246, 0.00042857142857142855, 0.0004489795918367347, 0.00046938775510204083, 0.0004897959183673469, 0.0005102040816326531, 0.0005306122448979592, 0.0005510204081632653, 0.0005714285714285715, 0.0005918367346938776, 0.0006122448979591836, 0.0006326530612244898, 0.0006530612244897959, 0.000673469387755102, 0.0006938775510204082, 0.0007142857142857143, 0.0007346938775510204, 0.0007

In [259]:
hyperparameter_tuning(algo="SGDRegressor()",param_grid=random_grid,n_iter=10,cv=5,verbose=2,random_state=42,X_train=X_train,Y_train=Y_train,X_test=X_test,Y_test=Y_test,evaluation_metric="r2_score")

Fitting 5 folds for each of 10 candidates, totalling 50 fits


/usr/local/lib/python3.9/dist-packages/sklearn/linear_model/_stochastic_gradient.py:1527: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  warnings.warn(
/usr/local/lib/python3.9/dist-packages/sklearn/linear_model/_stochastic_gradient.py:1527: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  warnings.warn(
/usr/local/lib/python3.9/dist-packages/sklearn/linear_model/_stochastic_gradient.py:1527: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  warnings.warn(
/usr/local/lib/python3.9/dist-packages/sklearn/linear_model/_stochastic_gradient.py:1527: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  warnings.warn(
/usr/local/lib/python3.9/dist-packages/sklearn/linear_model/_stochastic_

[CV 3/5] END copy_X=True, fit_intercept=True, positive=False;, score=-1491335583445575312343040.000 total time=   0.1s
[CV 4/5] END copy_X=True, fit_intercept=True, positive=True;, score=0.722 total time=   0.1s
[CV 5/5] END copy_X=True, fit_intercept=False, positive=False;, score=0.772 total time=   0.1s
[CV 1/5] END copy_X=False, fit_intercept=True, positive=False;, score=0.686 total time=   0.1s
[CV 2/5] END copy_X=False, fit_intercept=True, positive=True;, score=0.559 total time=   0.1s
[CV 3/5] END copy_X=False, fit_intercept=False, positive=False;, score=-1491335583445575312343040.000 total time=   0.1s
[CV 4/5] END copy_X=False, fit_intercept=False, positive=True;, score=0.722 total time=   0.1s
[CV 5/5] END copy_X=True, fit_intercept=True, positive=False;, score=0.772 total time=   0.1s
[CV 4/5] END criterion=poisson, max_depth=1000, max_features=sqrt, min_samples_leaf=8, min_samples_split=3, n_estimators=1000;, score=0.786 total time=   1.8s
[CV 4/5] END criterion=friedman_mse

/usr/local/lib/python3.9/dist-packages/sklearn/linear_model/_stochastic_gradient.py:1527: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  warnings.warn(
/usr/local/lib/python3.9/dist-packages/sklearn/linear_model/_stochastic_gradient.py:1527: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  warnings.warn(
/usr/local/lib/python3.9/dist-packages/sklearn/linear_model/_stochastic_gradient.py:1527: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  warnings.warn(
/usr/local/lib/python3.9/dist-packages/sklearn/linear_model/_stochastic_gradient.py:1527: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  warnings.warn(
/usr/local/lib/python3.9/dist-packages/sklearn/linear_model/_stochastic_

SGDRegressor() RandomCV Best Params : {'validation_fraction': 0.3938775510204082, 'tol': 0.007727272727272727, 'penalty': 'l2', 'n_iter_no_change': 2, 'max_iter': 13333, 'loss': 'squared_epsilon_insensitive', 'learning_rate': 'invscaling', 'l1_ratio': 0.08163265306122448, 'eta0': 0.05555555555555556, 'alpha': 0.0009183673469387755}
SGDRegressor() RandomCV Score: 0.7984591576471831
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Final score is  0.7848215372436905


In [260]:
# sgd=SGDRegressor()
# sgd_random_cv=RandomizedSearchCV(estimator=sgd,param_distributions=random_grid,n_iter=100,cv=5,verbose=0,random_state=100,n_jobs=-1)
# sgd_random_cv.fit(X_train,Y_train)

# best_params_=sgd_random_cv.best_params_
# print("RandomCV Best Params :",best_params_)
# best_random_grid=sgd_random_cv.best_estimator_
# y_pred6=best_random_grid.predict(X_test)
# score_6=r2_score(Y_test,y_pred6)
# print("RandomCV Score: ",score_6)

In [261]:
# param_grid6=defaultdict(list)
# for val in best_params_:
#     param_grid6[val].append(best_params_[val])
# print(param_grid6)

In [262]:
# model=SGDRegressor()
# grid_search=GridSearchCV(estimator=model,param_grid=param_grid6,cv=5,n_jobs=-1,verbose=3)
# grid_search.fit(X_train,Y_train)
# print(grid_search.best_estimator_)
# best_grid2=grid_search.best_estimator_
# final6=best_grid2.predict(X_test)
# score_final6=r2_score(Y_test,final6)
# print(score_final6)
# accuracies["SGDRegressor"]=score_final6

In [263]:
accuracies

{'LinearRegression()': -1.3643518980430038e+23,
 'RandomForestRegressor()': 0.8201522653525275,
 'SVR()': -7.943707625401863e-05,
 'KNeighborsRegressor()': 0.7073273687494936,
 'DecisionTreeRegressor()': 0.6699070971484998,
 'SGDRegressor()': 0.7848215372436905}

In [264]:
# 7 ElasticNet

In [265]:
alpha = [float(x) for x in np.linspace(0, 0.50,500)]
positive=['True','False']
l1_ratio=[float(x) for x in np.linspace(0, 0.5,100)]
max_iter=[int(x) for x in np.linspace(2000, 10000,200)]
tol=[float(x) for x in np.linspace(0.01, 0.001,200)]
warm_start = ['True','False']
selection=['cyclic','random']
copy_X=['True','False']

random_grid = {'copy_X':copy_X,
               'alpha':alpha,
               'l1_ratio': l1_ratio,
               'max_iter':max_iter,
               'tol':tol,
                'warm_start':warm_start,
                'selection':selection
               
               
                }
print(random_grid)

{'copy_X': ['True', 'False'], 'alpha': [0.0, 0.001002004008016032, 0.002004008016032064, 0.0030060120240480957, 0.004008016032064128, 0.00501002004008016, 0.0060120240480961915, 0.007014028056112224, 0.008016032064128256, 0.009018036072144287, 0.01002004008016032, 0.011022044088176352, 0.012024048096192383, 0.013026052104208416, 0.014028056112224447, 0.01503006012024048, 0.01603206412825651, 0.017034068136272545, 0.018036072144288574, 0.019038076152304607, 0.02004008016032064, 0.02104208416833667, 0.022044088176352703, 0.023046092184368736, 0.024048096192384766, 0.0250501002004008, 0.026052104208416832, 0.027054108216432865, 0.028056112224448895, 0.029058116232464928, 0.03006012024048096, 0.03106212424849699, 0.03206412825651302, 0.033066132264529056, 0.03406813627254509, 0.03507014028056112, 0.03607214428857715, 0.03707414829659318, 0.038076152304609215, 0.03907815631262525, 0.04008016032064128, 0.041082164328657314, 0.04208416833667334, 0.04308617234468937, 0.044088176352705406, 0.04

In [266]:
hyperparameter_tuning(algo="ElasticNet()",param_grid=random_grid,n_iter=10,cv=5,verbose=2,random_state=42,X_train=X_train,Y_train=Y_train,X_test=X_test,Y_test=Y_test,evaluation_metric="r2_score")

Fitting 5 folds for each of 10 candidates, totalling 50 fits
[CV 2/5] END copy_X=True, fit_intercept=True, positive=False;, score=0.559 total time=   0.1s
[CV 3/5] END copy_X=True, fit_intercept=True, positive=True;, score=-1491335583445575312343040.000 total time=   0.1s
[CV 4/5] END copy_X=True, fit_intercept=False, positive=False;, score=0.722 total time=   0.1s
[CV 5/5] END copy_X=True, fit_intercept=False, positive=True;, score=0.772 total time=   0.1s
[CV 1/5] END copy_X=False, fit_intercept=True, positive=True;, score=0.686 total time=   0.1s
[CV 2/5] END copy_X=False, fit_intercept=False, positive=False;, score=0.559 total time=   0.1s
[CV 3/5] END copy_X=False, fit_intercept=False, positive=True;, score=-1491335583445575312343040.000 total time=   0.1s
[CV 4/5] END copy_X=True, fit_intercept=True, positive=False;, score=0.722 total time=   0.1s
[CV 5/5] END criterion=poisson, max_depth=1000, max_features=sqrt, min_samples_leaf=8, min_samples_split=3, n_estimators=1000;, score=

In [267]:
# en=ElasticNet()
# en_random_cv=RandomizedSearchCV(estimator=en,param_distributions=random_grid,n_iter=100,cv=5,verbose=0,random_state=100,n_jobs=-1)
# en_random_cv.fit(X_train,Y_train)

# best_params_=en_random_cv.best_params_
# print("RandomCV Best Params :",best_params_)
# best_random_grid=en_random_cv.best_estimator_
# y_pred7=best_random_grid.predict(X_test)
# score_7=r2_score(Y_test,y_pred7)
# print("RandomCV Score: ",score_7)

In [268]:
# param_grid7=defaultdict(list)
# for val in best_params_:
#     param_grid7[val].append(best_params_[val])
# print(param_grid7)

In [269]:
# model=ElasticNet()
# grid_search=GridSearchCV(estimator=model,param_grid=param_grid7,cv=5,n_jobs=-1,verbose=3)
# grid_search.fit(X_train,Y_train)
# print(grid_search.best_estimator_)
# best_grid2=grid_search.best_estimator_
# final7=best_grid2.predict(X_test)
# score_final7=r2_score(Y_test,final7)
# print(score_final7)
# accuracies["ElasticNet"]=score_final7

In [270]:
# 8 Lasso

In [271]:
# alpha = [float(x) for x in np.linspace(0, 0.50,500)]
# positive=['True','False']
# l1_ratio=[float(x) for x in np.linspace(0, 0.5,100)]
# max_iter=[int(x) for x in np.linspace(2000, 10000,200)]
# tol=[float(x) for x in np.linspace(0.01, 0.001,200)]
# warm_start = ['True','False']
# selection=['cyclic','random']
# copy_X=['True','False']

# random_grid = {'copy_X':copy_X,
#                'alpha':alpha,
#                'l1_ratio': l1_ratio,
#                'max_iter':max_iter,
#                'tol':tol,
#                 'warm_start':warm_start,
#                 'selection':selection
               
               
#                 }
# print(random_grid)

In [272]:
max_iter=[int(x) for x in np.linspace(2000, 10000,500)]
tol=[float(x) for x in np.linspace(0.01, 0.001,300)]
warm_start = ['True','False']
selection=['cyclic','random']
alpha = [float(x) for x in np.linspace(0,0.50,500)]
# positive=['True','False']
copy_X=['True','False']
random_grid = {
               'alpha':alpha,
              
               'max_iter':max_iter,
               'tol':tol,
                'positive':positive,
                'copy_X':copy_X,
                'warm_start':warm_start,
                'selection':selection
               
               
                }
print(random_grid)


{'alpha': [0.0, 0.001002004008016032, 0.002004008016032064, 0.0030060120240480957, 0.004008016032064128, 0.00501002004008016, 0.0060120240480961915, 0.007014028056112224, 0.008016032064128256, 0.009018036072144287, 0.01002004008016032, 0.011022044088176352, 0.012024048096192383, 0.013026052104208416, 0.014028056112224447, 0.01503006012024048, 0.01603206412825651, 0.017034068136272545, 0.018036072144288574, 0.019038076152304607, 0.02004008016032064, 0.02104208416833667, 0.022044088176352703, 0.023046092184368736, 0.024048096192384766, 0.0250501002004008, 0.026052104208416832, 0.027054108216432865, 0.028056112224448895, 0.029058116232464928, 0.03006012024048096, 0.03106212424849699, 0.03206412825651302, 0.033066132264529056, 0.03406813627254509, 0.03507014028056112, 0.03607214428857715, 0.03707414829659318, 0.038076152304609215, 0.03907815631262525, 0.04008016032064128, 0.041082164328657314, 0.04208416833667334, 0.04308617234468937, 0.044088176352705406, 0.04509018036072144, 0.0460921843

In [273]:
hyperparameter_tuning(algo="Lasso()",param_grid=random_grid,n_iter=10,cv=5,verbose=2,random_state=42,X_train=X_train,Y_train=Y_train,X_test=X_test,Y_test=Y_test,evaluation_metric="r2_score")

Fitting 5 folds for each of 10 candidates, totalling 50 fits
[CV 4/5] END copy_X=True, fit_intercept=True, positive=False;, score=0.722 total time=   0.1s
[CV 1/5] END copy_X=True, fit_intercept=False, positive=False;, score=0.686 total time=   0.1s
[CV 2/5] END copy_X=True, fit_intercept=False, positive=True;, score=0.559 total time=   0.1s
[CV 2/5] END copy_X=False, fit_intercept=True, positive=False;, score=0.559 total time=   0.1s
[CV 3/5] END copy_X=False, fit_intercept=True, positive=True;, score=-1491335583445575312343040.000 total time=   0.1s
[CV 4/5] END copy_X=False, fit_intercept=False, positive=False;, score=0.722 total time=   0.1s
[CV 5/5] END copy_X=False, fit_intercept=False, positive=True;, score=0.772 total time=   0.1s
[CV 1/5] END criterion=poisson, max_depth=1000, max_features=sqrt, min_samples_leaf=8, min_samples_split=3, n_estimators=1000;, score=0.830 total time=   1.8s
[CV 3/5] END criterion=friedman_mse, max_depth=670, max_features=sqrt, min_samples_leaf=6, m

In [274]:
# lasso=Lasso()
# lasso_random_cv=RandomizedSearchCV(estimator=lasso,param_distributions=random_grid,n_iter=100,cv=5,verbose=0,random_state=100,n_jobs=-1)
# lasso_random_cv.fit(X_train,Y_train)

# best_params_=lasso_random_cv.best_params_
# print("RandomCV Best Params :",best_params_)
# best_random_grid=lasso_random_cv.best_estimator_
# y_pred8=best_random_grid.predict(X_test)
# score_8=r2_score(Y_test,y_pred8)
# print("RandomCV Score: ",score_8)

In [275]:
# param_grid8=defaultdict(list)
# for val in best_params_:
#     param_grid8[val].append(best_params_[val])
# print(param_grid8)

In [276]:
# param_grid8

In [277]:
# model=Lasso()
# grid_search=GridSearchCV(estimator=model,param_grid=param_grid8,cv=5,n_jobs=-1,verbose=3)
# grid_search.fit(X_train,Y_train)
# print(grid_search.best_estimator_)
# best_grid2=grid_search.best_estimator_
# final8=best_grid2.predict(X_test)
# score_final8=r2_score(Y_test,final8)
# print(score_final8)
# accuracies["Lasso"]=score_final8

In [278]:
# 9 Ridge

In [279]:
max_iter=[int(x) for x in np.linspace(2000, 10000,200)]
tol=[float(x) for x in np.linspace(0.01, 0.001,200)]
alpha = [float(x) for x in np.linspace(0.1,0.50,400)]
positive=['False']
copy_X=['True','False']
solver=["auto"]

random_grid = {
               'alpha':alpha,
              
               'max_iter':max_iter,
               'tol':tol,
                'positive':positive,
                'copy_X':copy_X,
                'solver':solver
               
               
                }
print(random_grid)

{'alpha': [0.1, 0.10100250626566416, 0.10200501253132832, 0.10300751879699249, 0.10401002506265665, 0.10501253132832081, 0.10601503759398496, 0.10701754385964912, 0.1080200501253133, 0.10902255639097745, 0.11002506265664161, 0.11102756892230577, 0.11203007518796992, 0.1130325814536341, 0.11403508771929825, 0.11503759398496241, 0.11604010025062657, 0.11704260651629073, 0.1180451127819549, 0.11904761904761905, 0.12005012531328321, 0.12105263157894737, 0.12205513784461153, 0.1230576441102757, 0.12406015037593986, 0.12506265664160401, 0.12606516290726819, 0.12706766917293233, 0.1280701754385965, 0.12907268170426067, 0.13007518796992482, 0.131077694235589, 0.13208020050125313, 0.1330827067669173, 0.13408521303258147, 0.13508771929824562, 0.1360902255639098, 0.13709273182957393, 0.1380952380952381, 0.13909774436090228, 0.14010025062656642, 0.1411027568922306, 0.14210526315789473, 0.1431077694235589, 0.14411027568922308, 0.14511278195488722, 0.1461152882205514, 0.14711779448621554, 0.14812030

In [280]:
hyperparameter_tuning(algo="Ridge()",param_grid=random_grid,n_iter=10,cv=5,verbose=2,random_state=42,X_train=X_train,Y_train=Y_train,X_test=X_test,Y_test=Y_test,evaluation_metric="r2_score")

Fitting 5 folds for each of 10 candidates, totalling 50 fits
Ridge() RandomCV Best Params : {'tol': 0.007738693467336683, 'solver': 'auto', 'positive': 'False', 'max_iter': 3567, 'copy_X': 'False', 'alpha': 0.3917293233082707}
Ridge() RandomCV Score: 0.8350408062056907
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Final score is  0.8350408062056907


In [281]:
# ridge=Ridge()
# ridge_random_cv=RandomizedSearchCV(estimator=ridge,param_distributions=random_grid,n_iter=100,cv=5,verbose=0,random_state=100,n_jobs=-1)
# ridge_random_cv.fit(X_train,Y_train)

# best_params_=ridge_random_cv.best_params_
# print("RandomCV Best Params :",best_params_)
# best_random_grid=ridge_random_cv.best_estimator_
# y_pred9=best_random_grid.predict(X_test)
# score_9=r2_score(Y_test,y_pred9)
# print("RandomCV Score: ",score_9)

In [282]:
# param_grid9=defaultdict(list)
# for val in best_params_:
#     param_grid9[val].append(best_params_[val])
# print(param_grid9)

In [283]:
# model=Ridge()
# grid_search=GridSearchCV(estimator=model,param_grid=param_grid9,cv=5,n_jobs=-1,verbose=3)
# grid_search.fit(X_train,Y_train)
# print(grid_search.best_estimator_)
# best_grid2=grid_search.best_estimator_
# final9=best_grid2.predict(X_test)
# score_final9=r2_score(Y_test,final9)
# print(score_final9)
# accuracies["Ridge"]=score_final9

In [284]:
accuracies

{'LinearRegression()': -1.3643518980430038e+23,
 'RandomForestRegressor()': 0.8201522653525275,
 'SVR()': -7.943707625401863e-05,
 'KNeighborsRegressor()': 0.7073273687494936,
 'DecisionTreeRegressor()': 0.6699070971484998,
 'SGDRegressor()': 0.7848215372436905,
 'ElasticNet()': 0.8061969923154417,
 'Lasso()': 0.7442522278615189,
 'Ridge()': 0.8350408062056907}

In [ ]:

# sfs1 = SFS(RandomForestRegressor(), 
#                k_features=(3,len(X_train.columns)), 
#                forward=True, 
#                floating=False, 
#                verbose=1,
#                scoring='r2',
#                cv=5,
#                n_jobs=-1)
# sfs1=sfs1.fit(X_train, Y_train)
# print("sfs1 max score features are :",sfs1.k_feature_names_)
# print("sfs1 max score is :",sfs1.k_score_)
# print()
# model_=RandomForestRegressor()
# model_.fit(X_train.loc[:,list(sfs1.k_feature_names_)],Y_train)
# score=model_.score(X_test.loc[:,list(sfs1.k_feature_names_)],Y_test)
# print("Prediction using selected features is :",score)
# accuracies["RandomForestRegressor"]=score

In [ ]:
# sfs1 = SFS(SVR(), 
#            k_features=(3,len(X_train.columns)), 
#            forward=True, 
#            floating=False, 
#            verbose=1,
#            scoring='r2',
#            cv=5,
#            n_jobs=-1)
# sfs1=sfs1.fit(X_train, Y_train)
# print("sfs1 max score features are :",sfs1.k_feature_names_)
# print("sfs1 max score is :",sfs1.k_score_)
# print()
# model_=SVR()
# model_.fit(X_train.loc[:,list(sfs1.k_feature_names_)],Y_train)
# score=model_.score(X_test.loc[:,list(sfs1.k_feature_names_)],Y_test)
# print("Prediction using selected features is :",score)
# accuracies["SVR"]=score

In [ ]:
# sfs1 = SFS(KNeighborsRegressor(), 
#            k_features=(3,len(X_train.columns)), 
#            forward=True, 
#            floating=False, 
#            verbose=1,
#            scoring='r2',
#            cv=5,
#            n_jobs=-1)
# sfs1=sfs1.fit(X_train, Y_train)
# print("sfs1 max score features are :",sfs1.k_feature_names_)
# print("sfs1 max score is :",sfs1.k_score_)
# print()
# model_=KNeighborsRegressor()
# model_.fit(X_train.loc[:,list(sfs1.k_feature_names_)],Y_train)
# score=model_.score(X_test.loc[:,list(sfs1.k_feature_names_)],Y_test)
# print("Prediction using selected features is :",score)
# accuracies["KNeighborsRegressor"]=score

In [ ]:
# sfs1 = SFS(DecisionTreeRegressor(), 
#                k_features=(3,len(X_train.columns)), 
#                forward=True, 
#                floating=False, 
#                verbose=1,
#                scoring='r2',
#                cv=5,
#                n_jobs=-1)
# sfs1=sfs1.fit(X_train, Y_train)
# print("sfs1 max score features are :",sfs1.k_feature_names_)
# print("sfs1 max score is :",sfs1.k_score_)
# print()
# model_=DecisionTreeRegressor()
# model_.fit(X_train.loc[:,list(sfs1.k_feature_names_)],Y_train)
# score=model_.score(X_test.loc[:,list(sfs1.k_feature_names_)],Y_test)
# print("Prediction using selected features is :",score)
# accuracies["DecisionTreeRegressor"]=score

In [ ]:
# sfs1 = SFS(SGDRegressor(), 
#                k_features=(3,len(X_train.columns)), 
#                forward=True, 
#                floating=False, 
#                verbose=1,
#                scoring='r2',
#                cv=5,
#                n_jobs=-1)
# sfs1=sfs1.fit(X_train, Y_train)
# print("sfs1 max score features are :",sfs1.k_feature_names_)
# print("sfs1 max score is :",sfs1.k_score_)
# print()
# model_=SGDRegressor()
# model_.fit(X_train.loc[:,list(sfs1.k_feature_names_)],Y_train)
# score=model_.score(X_test.loc[:,list(sfs1.k_feature_names_)],Y_test)
# print("Prediction using selected features is :",score)
# accuracies["SGDRegressor"]=score

In [174]:
# sfs1 = SFS(ElasticNet(), 
#                k_features=(3,len(X_train.columns)), 
#                forward=True, 
#                floating=False, 
#                verbose=1,
#                scoring='r2',
#                cv=5,
#                n_jobs=-1)
# sfs1=sfs1.fit(X_train, Y_train)
# print("sfs1 max score features are :",sfs1.k_feature_names_)
# print("sfs1 max score is :",sfs1.k_score_)
# print()
# model_=ElasticNet()
# model_.fit(X_train.loc[:,list(sfs1.k_feature_names_)],Y_train)
# score=model_.score(X_test.loc[:,list(sfs1.k_feature_names_)],Y_test)
# print("Prediction using selected features is :",score)
# accuracies["ElasticNet"]=score

In [214]:
# sfs1 = SFS(Lasso(), 
#                k_features=(3,len(X_train.columns)), 
#                forward=True, 
#                floating=False, 
#                verbose=1,
#                scoring='r2',
#                cv=5,
#                n_jobs=-1)
# sfs1=sfs1.fit(X_train, Y_train)
# print("sfs1 max score features are :",sfs1.k_feature_names_)
# print("sfs1 max score is :",sfs1.k_score_)
# print()
# model_=Lasso()
# model_.fit(X_train.loc[:,list(sfs1.k_feature_names_)],Y_train)
# score=model_.score(X_test.loc[:,list(sfs1.k_feature_names_)],Y_test)
# print("Prediction using selected features is :",score)
# accuracies["Lasso"]=score

In [ ]:
# sfs1 = SFS(Ridge(), 
#                k_features=(3,len(X_train.columns)), 
#                forward=True, 
#                floating=False, 
#                verbose=1,
#                scoring='r2',
#                cv=5,
#                n_jobs=-1)
# sfs1=sfs1.fit(X_train, Y_train)
# print("sfs1 max score features are :",sfs1.k_feature_names_)
# print("sfs1 max score is :",sfs1.k_score_)
# print()
# model_=Ridge()
# model_.fit(X_train.loc[:,list(sfs1.k_feature_names_)],Y_train)
# score=model_.score(X_test.loc[:,list(sfs1.k_feature_names_)],Y_test)
# print("Prediction using selected features is :",score)
# accuracies["Ridge"]=score

In [ ]:
accuracies